In [7]:
import wandb
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import sys
import pathlib
import os
import numpy as np

from clifford.models.modules.linear import MVLinear
from clifford.models.modules.gp import SteerableGeometricProductLayer
from clifford.models.modules.mvlayernorm import MVLayerNorm
from clifford.models.modules.mvsilu import MVSiLU
from clifford.algebra.cliffordalgebra import CliffordAlgebra

from image2sphere.predictor import I2S
from image2sphere.models import ResNet
import image2sphere.pascal_dataset
from image2sphere.models import ResNet
from image2sphere.pascal_dataset import Pascal3D
from image2sphere.so3_utils import so3_healpix_grid

from src.model import MLPBaseline, TralaleroCompetitor
from src.dataset import PascalSanityCheckDataset
from src.train_utils import train_epoch
from src.evaluation_metrics import calculate_evaluation_metrics

from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [8]:
import torch
from src.evaluation_metrics import project_to_orthogonal_manifold, create_technical_matrices, acc_at
from dataclasses import dataclass
@dataclass
class configin:
    batch_size = 32
    path_to_datasets = "/Users/grigorychaykovsky/Downloads"
    device = torch.device("mps")
    sanity_check = False



config = configin()
create_technical_matrices(config)

pass

In [9]:


model = I2S(encoder='resnet101', eval_grid_rec_level=3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.load_state_dict(
    torch.load('/Users/grigorychaykovsky/Downloads/pascal3d_checkpoint.pt', map_location=device)['model_state_dict'],
)
model.eval()

I2S(
  (encoder): ResNet(
    (layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (downs

In [7]:
from image2sphere.so3_utils import rotation_error

dataset = PascalSanityCheckDataset(config)

loader = DataLoader(dataset, batch_size=32)


In [10]:
from src.dataset import create_dataloaders

train_loader, val_loader = create_dataloaders(config)


In [19]:
from src.train_utils import load_checkpoint

checkpoint = pathlib.Path("/Users/grigorychaykovsky/Downloads/Vanilla_MLP_v0.pth")

model = MLPBaseline()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1.0)
model, optimizer, scheduler = load_checkpoint(model, optimizer, scheduler, checkpoint, config.device)

In [22]:
err = calculate_evaluation_metrics(model, val_loader, config)

Evaluating Model: 100%|██████████| 41/41 [00:47<00:00,  1.15s/it]


In [25]:
np.median(err)

np.float32(47.522896)

In [29]:
checkpoint = pathlib.Path("/Users/grigorychaykovsky/Downloads/Vanilla_Tralalero_v0.pth")

model = TralaleroCompetitor(CliffordAlgebra((1, 1, 1, 1, -1)))
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1.0)
model, optimizer, scheduler = load_checkpoint(model, optimizer, scheduler, checkpoint, config.device)

In [30]:
err = calculate_evaluation_metrics(model, val_loader, config)

Evaluating Model:   0%|          | 0/41 [00:00<?, ?it/s]/Users/grigorychaykovsky/Library/Caches/pypoetry/virtualenvs/3d-pose-estimation-exps-UtNqBTXH-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
Evaluating Model: 100%|██████████| 41/41 [00:45<00:00,  1.12s/it]


In [33]:
np.median(err)

np.float32(103.93534)

In [11]:
err = calculate_evaluation_metrics(model, val_loader, config)

Evaluating Model:   0%|          | 0/41 [00:00<?, ?it/s]/Users/grigorychaykovsky/Library/Caches/pypoetry/virtualenvs/3d-pose-estimation-exps-UtNqBTXH-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
Evaluating Model: 100%|██████████| 41/41 [00:45<00:00,  1.10s/it]


In [ ]:
# import wandb

# run = wandb.init(entity="chagrygoris", project="Tralalero", name="I2S_BASELINE")

# for _ in range(15):
#     run.log({
#         "median_rotation_error" : np.median(err).__float__()
#     })

# run.finish()

In [ ]:
X = pred_rotmat[1]
torch.det(X)
u, s, v = torch.svd(X)
I = torch.eye(3)
I[-1, -1] = torch.det(u @ v)
(u @ I @ v) == X

torch.allclose(u @ v.T, X)

True

In [ ]:
res = (np.stack(errors) * 180 / np.pi) <= 15
res.mean()

np.float64(0.75)

In [ ]:
# !wget ftp://cs.stanford.edu/cs/cvgl/PASCAL3D+_release1.1.zip

In [28]:
algebra = CliffordAlgebra((1, 1, 1, 1, -1))
gp = SteerableGeometricProductLayer(algebra, 1)
x = torch.zeros(1, 1, 32)
x[-1] = 100500
xidx = torch.arange(0, 32, dtype=torch.long)

activ = MVSiLU(algebra=algebra, channels=1)
mv = algebra.embed(x, xidx)
mv = gp(mv)
mv = activ(mv)
mv.size()

torch.Size([1, 1, 32])

In [ ]:
from src.model import TralaleroTralala

model = TralaleroTralala(algebra)
model(mv).size()

torch.Size([1, 9, 32])

In [ ]:


rn = ResNet()
baseline = MLPBaseline()
tralalero = TralaleroCompetitor(algebra)


In [ ]:



dataset = PascalSanityCheckDataset(config)
loader = DataLoader(dataset, shuffle=True, batch_size=32)

optimizer = torch.optim.Adam(baseline.parameters())
criterion = nn.MSELoss()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/grigorychaykovsky/Downloads/PASCAL3D+_release1.1/PASCAL/VOCdevkit/VOC2012/ImageSets/Main/aeroplane_train.txt'

In [ ]:
mse_baseline = train_epoch(baseline, loader, optimizer, criterion)

100%|██████████| 1/1 [00:06<00:00,  6.81s/it]


In [ ]:
dataset = PascalSanityCheckDataset()
loader = DataLoader(dataset, shuffle=True, batch_size=32)

optimizer = torch.optim.Adam(tralalero.parameters())
criterion = nn.MSELoss()

In [ ]:
mse_tralalero = train_epoch(tralalero, loader, optimizer, criterion)

100%|██████████| 1/1 [00:06<00:00,  6.64s/it]


In [ ]:
print("MSE on 1 batch for MLPBaseline is {}".format(mse_baseline))
print("MSE on 1 batch for Tralalero architecture is {}".format(mse_tralalero))

MSE on 1 batch for MLPBaseline is 0.01835789903998375
MSE on 1 batch for Tralalero architecture is 0.013551775366067886


## CGENN basic blocks

In [ ]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()

        self.layers = nn.Sequential(
            MVLinear(algebra, in_features, out_features),
            MVSiLU(algebra, out_features),
            SteerableGeometricProductLayer(algebra, out_features),
            MVLayerNorm(algebra, out_features)
        )

    def forward(self, input):
        # [batch_size, in_features, 2**d] -> [batch_size, out_features, 2**d]
        return self.layers(input)

class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers=2):
        super().__init__()

        layers = []
        for i in range(n_layers - 1):
            layers.append(
                CGEBlock(algebra, in_features, hidden_features)
            )
            in_features = hidden_features

        layers.append(
            CGEBlock(algebra, hidden_features, out_features)
        )
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

## CGENN model

In [ ]:
class EquvariantCGENN(nn.Module):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.cgemlp = CGEMLP(ca, in_features, hidden_features, hidden_features)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features)
        )

    def forward(self, input):
        h = self.cgemlp(input)
        # Index the hidden states at 0 to get the invariants, and let a regular MLP do the final processing.
        return self.mlp(h[..., 0])

# Final models

In [ ]:
class IGA2S(nn.Module):
    def __init__(
        self,
        num_classes, # provided as part of the input
        train_grid_rec_level,
        eval_grid_rec_level,
        size=50,
        pretrained=True,
        global_pooling=True,
        hidden_dim=42
    ):

        self.encoder = ResNet(size, pretrained, global_pooling)

        train_grid = so3.healpix_grid(rec_level=train_grid_rec_level)
        self.register_buffer('train_rotmats', o3.angles_to_matrix(*train_grid))


        eval_grid = so3.healpix_grid(rec_level=eval_grid_rec_level)
        self.register_buffer('eval_rotmats', o3.angles_to_matrix(*eval_grid))

        input_dim = self.encoder.output_shape[0]
        self.equiv = EquvariantCGENN(input_dim + num_classes, hidden_dim, )
